#Comparison Models for LiverNet

In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

Using TensorFlow backend.


##Defining required global variables

In [13]:
train_path = '/random_crop_kmc_data/Train/'
val_path = '/random_crop_kmc_data/Validation/'
test_path = '/random_crop_kmc_data/Test/'
batch_size = 32
img_height = 224
img_width = 224
input_shape = (img_height , img_width , 3)

#chose mdoel name out of 'resnet50','densenet121','mobilenetv2','inceptionresnetv2'
model_name = 'inceptionresnetv2'

##Reading Data and Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    horizontal_flip = True,
    vertical_flip = True,
    featurewise_std_normalization=True)

valid_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True,
    class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True,
    class_mode='categorical')

##Defining Comparison Models

In [ ]:
def resnet50():

  base_model = tf.keras.applications.ResNet50(weights = None , include_top = False , input_shape=(224,224,3))
  x = base_model.output
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  pred = tf.keras.layers.Dense(4 , activation='softmax')(x)
  model = tf.keras.models.Model(inputs = base_model.input , outputs = pred)
  return model 

def densenet121():

  base_model = tf.keras.applications.DenseNet121(weights = None , include_top = False , input_shape=(224,224,3))
  x = base_model.output
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  pred = tf.keras.layers.Dense(4 , activation='softmax')(x)
  model = tf.keras.models.Model(inputs = base_model.input , outputs = pred)
  return model 

def mobilenetv2():

  base_model = tf.keras.applications.MobileNetV2(weights = None , include_top = False , input_shape=(224,224,3))
  x = base_model.output
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  pred = tf.keras.layers.Dense(4 , activation='softmax')(x)
  model = tf.keras.models.Model(inputs = base_model.input , outputs = pred)
  return model 

def inceptionresnetv2():

  base_model = tf.keras.applications.InceptionResNetV2(weights = None , include_top = False , input_shape=(224,224,3))
  x = base_model.output
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  pred = tf.keras.layers.Dense(4 , activation='softmax')(x)
  model = tf.keras.models.Model(inputs = base_model.input , outputs = pred)
  return model 


model = None

if(model_name == 'resnet50'):
  model =  resnet50()
elif(model_name == 'densenet121'):
  model = densenet121()
elif(model_name == 'mobilenetv2'):
  model = mobilenetv2()
elif(model_name == 'inceptionresnetv2'):
  model = inceptionresnetv2()


adam = tf.keras.optimizers.Adam(lr = 0.00001)
model.compile(optimizer = adam , loss = 'categorical_crossentropy' , metrics = ["acc"])
model.summary()

##Training

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss' ,
                                                  factor = 0.1 , patience = 3 , verbose=1 , cooldown = 1)

history = model.fit_generator(train_generator , validation_data = validation_generator , 
                                  steps_per_epoch= len(train_generator) ,
                                  validation_steps = len(validation_generator)
                                  ,epochs = 40 , callbacks = [reduce_lr] )

##Plots for Accuracy anf loss 

In [ ]:
plt.plot(history.history['loss'] , label = 'train_loss')
plt.plot(history.history['val_loss'] , label = 'val_loss')
plt.legend
plt.show()

plt.plot(history.history['acc'] , label = 'train_acc')
plt.plot(history.history['val_acc'] , label = 'val_acc')
plt.legend
plt.show()

##Testing

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=1,
    shuffle = False,
    class_mode='categorical')
print('original')
print(test_generator.labels)
print('\n')
print('predicted')
test_step = test_generator.n//test_generator.batch_size
test_generator.reset()
pred = model_deep.predict_generator(test_generator , steps = test_step , verbose = 1)
pred_class_indices = np.argmax(pred,axis=1)
print(pred_class_indices)
print('\n')

##Result Analysis

In [ ]:
from sklearn.metrics import accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score,jaccard_score,classification_report
from sklearn.metrics import precision_score,recall_score,jaccard_score


## Classwise performance analysis

classes = [0,1,2,3]


for cl in classes:

    print("class: ",cl)

    a1 = np.uint8(test_generator.labels == cl)
    a2 = np.uint8(pred_class_indices == cl)

    print('Accuracy {}'.format(accuracy_score(y_true=a1, y_pred=a2)))
    print('F1 {}'.format(f1_score(y_true=a1, y_pred=a2)))
    print('precision {}'.format(precision_score(y_true=a1, y_pred=a2)))
    print('recall {}'.format(recall_score(y_true=a1, y_pred=a2)))

    print('jaccard {}'.format(jaccard_score(y_true=a1, y_pred=a2)))
    print("_______________________________")


## Overall performance analysis

print('Accuracy {}'.format(accuracy_score(y_true=test_generator.labels, y_pred=pred_class_indices)))
print('F1 {}'.format(f1_score(y_true=test_generator.labels, y_pred=pred_class_indices,average = "macro")))
print('precision {}'.format(precision_score(y_true=test_generator.labels, y_pred=pred_class_indices,average = "macro")))
print('recall {}'.format(recall_score(y_true=test_generator.labels, y_pred=pred_class_indices,average = "macro")))

print('jaccard {}'.format(jaccard_score(y_true=test_generator.labels, y_pred=pred_class_indices,average = "macro")))
print('confusion_matrix\n {}'.format(confusion_matrix(y_true=test_generator.labels, y_pred=pred_class_indices)))
print('classification_report\n {}'.format(classification_report(y_true=test_generator.labels, y_pred=pred_class_indices)))
print('\n\n')